In [1]:
import pandas as pd
from os import listdir


In [2]:
file_dir = '../../data_csse/csse_covid_19_data/csse_covid_19_daily_reports'
files_present = [f for f in listdir(file_dir) if '2021' in f]

In [3]:
# we know that the file system is in mm-dd-yyyy format (terrible convention)
dates = pd.date_range(start='2021-10-15',end='2022-01-31')  # already set period of interest
dates_us = dates.strftime('%m-%d-%Y')  # convert to US method for matching
dates_filenames = [f"{n}.csv" for n in dates_us]
# sanity check
match = [f in files_present for f in dates_filenames]
print(f"All files present?: {all(match)}")

All files present?: False


In [4]:
# targeted extraction for SA and for dates of interest
sa_dict = {}
for fname, date in zip(dates_filenames,dates):  # assume correct order
    readfile = pd.read_csv(f'{file_dir}/{fname}')
    sa_dict[date] = readfile[readfile['Country_Region'] == 'South Africa'].squeeze()

del readfile, fname, date

cases_df = pd.DataFrame.from_dict(sa_dict, orient='index')
del sa_dict

# Correction
a manual correction is added to the SA daily case rates
[link, Twitter](https://twitter.com/nicd_sa/status/1463200722615513093)
thus 23 nov 2021 --> 868 (instead of 18586
(tbvh, many annoying things happening getting to here)

In [5]:
# correct day 23 Nov 2021 to be 868
#
parse1 = cases_df['Confirmed'].diff(periods=1)

parse1.loc['2021-11-23'] = 868 # apply correction

In [6]:
parse1 = parse1.dropna().astype(int)

In [7]:
save_to_dir = "../data_input"
save_to_name = "data_CSSE.pickletable"
parse1.to_pickle(f"{save_to_dir}/{save_to_name}")

In [8]:
# readfile = pd.read_csv(f'{file_dir}/{file_name}')
# # truncate for SA already
# readfile = readfile[readfile['Country/Region'] == 'South Africa'].drop(['Province/State','Country/Region','Lat','Long'], axis=1).T.squeeze().rename('cases')
# readfile.index = pd.to_datetime(readfile.index)